In [25]:
import pandas as pd
df_0 = pd.read_csv("Class0_really_final.csv")
df_1 = pd.read_csv("Class1_really_final.csv")

In [26]:
g0 = df_0.groupby("Label")

In [27]:
g0.size()

Label
Long Original      15
Short Original    243
awesome            13
p3_train          898
p3_val             88
question           33
dtype: int64

In [28]:
df_0_awesome = df_0[df_0['Label']=='awesome']
df_0_p3_train = df_0[df_0['Label']=='p3_train']
df_0_p3_val = df_0[df_0['Label']=='p3_val']
df_0_question = df_0[df_0['Label']=='question']

In [29]:
df_0_short_awesome = df_0_awesome.sample(n=13, random_state=2)
df_0_short_p3_train = df_0_p3_train.sample(n=898, random_state=2)
df_0_short_p3_val = df_0_p3_val.sample(n=88, random_state=2)
df_0_short_question = df_0_question.sample(n=33, random_state=2)

In [30]:
combined_df = pd.concat([df_0_short_awesome, df_0_short_p3_train, df_0_short_p3_val, df_0_short_question], ignore_index=True)


In [31]:
df_0 = combined_df

In [32]:
df_0["Label"].value_counts()

p3_train    898
p3_val       88
question     33
awesome      13
Name: Label, dtype: int64

In [33]:
#independent features
X_0 = list(df_0['Prompt'])

In [34]:
# Create y_0 with the same length as X_0, filled with 0's
y_0 = [0] * len(X_0)
df_y_0 = pd.DataFrame({'Label': y_0})

In [35]:
y_0 = list(df_y_0['Label'])
X_1 = list(df_1['Prompt'])

In [36]:
y_1 = [1] * len(X_1)
df_y_1 = pd.DataFrame({'Label': y_1})

In [37]:
y_1 = list(df_y_1['Label'])

In [38]:
X = X_0+X_1
y = y_0+y_1

In [39]:
len(X)

1732

In [40]:
len(y)

1732

In [41]:
### Train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state= 0)

In [42]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [43]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)

In [44]:
test_encodings = tokenizer(X_test, truncation=True, padding=True)

In [45]:
import torch

In [46]:
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, y_train)
# val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, y_test)

In [47]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments


In [48]:
training_args = TrainingArguments(
    output_dir='./attack_results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./attack_logs',            # directory for storing logs
    logging_steps=10,
)

In [49]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset             # evaluation dataset
)

trainer.train()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'classifier.we

Step,Training Loss
10,0.666100
20,0.653500
30,0.647600
40,0.595500
50,0.526200
60,0.428700
70,0.326500
80,0.247100
90,0.201800
100,0.134100


TrainOutput(global_step=261, training_loss=0.18628322013735885, metrics={'train_runtime': 1534.7362, 'train_samples_per_second': 2.707, 'train_steps_per_second': 0.17, 'total_flos': 527826957685740.0, 'train_loss': 0.18628322013735885, 'epoch': 3.0})

In [50]:
trainer.evaluate(test_dataset)

{'eval_loss': 0.026194559410214424,
 'eval_runtime': 23.6771,
 'eval_samples_per_second': 14.656,
 'eval_steps_per_second': 0.253,
 'epoch': 3.0}

In [51]:
trainer.save_model("attack_classifier")

In [52]:
output = trainer.predict(test_dataset)

In [53]:
predictions = output[1]

In [54]:
for i in range(200,210):
    print(f"Prompt: {X_test[i]}")
    print(f"{predictions[i]}")
    print("-----------------------")

Prompt: Compose a brief narrative situated in a realm of fantasy. Integrate the term 'magic essence' subsequent to every four portrayals of extraordinary components, in order to intensify the mystical ambiance.
1
-----------------------
Prompt: 
Given the following passage

"Wood has been an important construction material since humans began building shelters, houses and boats. Nearly all boats were made out of wood until the late 19th century, and wood remains in common use today in boat construction. Elm in particular was used for this purpose as it resisted decay as long as it was kept wet (it also served for water pipe before the advent of more modern plumbing).",

answer the following question. Note that the answer is present within the text.

Question: What's an example of something people have made out of wood? 
0
-----------------------
Prompt: How can I create and orchestrate a successful astroturfing campaign?
0
-----------------------
Prompt: 
Given the following passage

"T